In [2]:
import numpy as np
import xml.etree.ElementTree as ET
import napari
import tifffile as tiff

In [3]:
def print_metadata_structure(element, indent=0):
    print("  " * indent + f"Tag: {element.tag}")
    if element.attrib:
        print("  " * (indent + 1) + "Attributes:")
        for key, value in element.attrib.items():
            print("  " * (indent + 2) + f"{key}: {value}")
    for child in element:
        print_metadata_structure(child, indent + 1)

In [1]:
ome_path = '/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/cycif_ovca/img/S012_chemonaive.ome.tif'
#img = tiff.imread(ome_path)
#seg = tiff.imread('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/IMMUcan/segmentation/IMMUcan_2022_WFLOW_10034294-GI-VAR-TIS-UNST-03_004.tiff')

In [ ]:
with tiff.TiffFile(ome_path) as tif:
    img = tif.asarray()
    print(img.shape)

In [6]:
with tiff.TiffFile(ome_path) as tif:
    ome_metadata = tif.ome_metadata
    
    if ome_metadata:
        root = ET.fromstring(ome_metadata)
        print("Complete OME Metadata Structure:")
        print_metadata_structure(root)
    else:
        print("No OME metadata found in the file.")

Complete OME Metadata Structure:
Tag: {http://www.openmicroscopy.org/Schemas/OME/2016-06}OME
  Attributes:
    UUID: urn:uuid:dda10948-b470-434a-aa4b-45c6e3e005a9
    {http://www.w3.org/2001/XMLSchema-instance}schemaLocation: http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd
  Tag: {http://www.openmicroscopy.org/Schemas/OME/2016-06}Image
    Attributes:
      ID: Image:0
    Tag: {http://www.openmicroscopy.org/Schemas/OME/2016-06}Pixels
      Attributes:
        BigEndian: false
        DimensionOrder: XYZCT
        ID: Pixels:0
        PhysicalSizeX: 0.649999976158
        PhysicalSizeXUnit: µm
        PhysicalSizeY: 0.649999976158
        PhysicalSizeYUnit: µm
        SizeC: 52
        SizeT: 1
        SizeX: 35716
        SizeY: 15888
        SizeZ: 1
        Type: uint16
      Tag: {http://www.openmicroscopy.org/Schemas/OME/2016-06}Channel
        Attributes:
          ID: Channel:0:0
          Name: Channel 0
          Samp

In [4]:
with tiff.TiffFile('/Users/lukashat/Documents/PhD_Schapiro/Projects/phenotype_benchmark/datasets/cycif_ovca/img/S012_chemonaive.ome.tif') as tif:
    md = tif.ome_metadata
    #channel_names = [channel.get('Name') for channel in tif.md.findall('.//Channel')]
    root = ET.fromstring(md)
    ns = {'ome': 'http://www.openmicroscopy.org/Schemas/ome/2013-06s'}
    channels = root.findall('ome:Image/ome:Pixels/ome:Channel', ns)
    channel_names = [channel.get('Name') for channel in channels]

In [5]:
channel_names

[]

In [ ]:
Viewer = napari.Viewer()
Viewer.add_image(img)
Viewer.add_image(seg)

In [ ]:
channel_names

In [ ]:
with open('/Volumes/Lukas_SSD/phenotyping_benchmark/datasets/breast_fibro_IMC/markers.txt', 'w', encoding='utf-8') as f:
    f.writelines([str(i) + '\n' for i in channel_names])

In [ ]:
seg

In [ ]:
array = np.array(seg, dtype=np.uint32)
array.max()

In [ ]:
array

In [ ]:
len(np.unique(array))